<h1>Линейная алгебра: Проект</h1>

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, следует обосновать корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

**Описание данных**

    - Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    - Целевой признак: количество страховых выплат клиенту за последние 5 лет.

**План работы:** первым делом мы ознакомимся с данными, затем - попробуем умножить признаки на обратимую матрицу и посмотрим, что получится; далее, мы подберем удовлетворяющий условиям задачи алгоритм и проверим его.

<h3>Оглавление</h3>

1. [Шаг 1: Изучение данных](#start)
2. [Шаг 2: Умножение признаков на обратимую матрицу](#mult)
3. [Шаг 3: Выбор алгоритма преобразования](#alg)
4. [Шаг 4: Проверка и программирование выбранного алгоритма](#alg1)

<h3>Шаг 1: Изучение данных</h3>
<a id='start'></a>

Первым делом импортируем необходимые нам библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Прочитаем данные и посмотрим на первые пять строк.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Посмотрим на общую информацию.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Все выглядит отлично, но взглянем на всякий случай и на результат работы метода describe - так мы обнаружим явные аномалии, если они есть.

In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Вывод по Шагу 1:** на данном шаге мы прочитали датафрейм, предоставленный нам для работы, а также изучили его: в нашем распоряжении таблица, состоящая из 5000 строк и 5 столбцов. Все столбцы в таблице имеют численный тип переменных - int или float - что отлично соответствует задаче, с данными все в порядке - аномалий и пропусков нет. Можно приступать к основным действиям.

<h3> Шаг 2: Умножение признаков на обратимую матрицу</h3>
<a id='mult'></a>

Ответим на вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Поставим в формулу обучения PX вместо X. Будем считать что матрица P - обратимая.

$$
w = ((XP)^T XP)^{-1} (XP)^T y = (XP)^{-1}((XP)^{T})^{-1}(XP)^T y = (XP)^{-1} E y = (XP)^{-1} y
$$

Получили такое значение весов. Подставим веса в формулу предсказания, при условии, что предсказываем для той же выборки, на которой и обучались. Также учтем, что признаки - X - поданные в функцию предсказания также преобразованы - умножены на матрицу P.

$$
a = Xw = XP (XP)^{-1}y = y
$$

Получили значения целевого признака, ровно как и должно быть - что и следовало доказать.

**Вывод по Шагу 2**: при умножении признаков на обратимую матрицу качество модели не изменится, при условии, что признаки, для которых в дальнейшем совершаются предсказания, также предварительно умножены на эту обратимую матрицу. 

<h3> Шаг 3: Выбор алгоритма преобразования</h3>
<a id='alg'></a>

В этом разделе нам следует предложить алгоритм преобразования данных для решения задачи. 

Ранее мы выяснили, что умножение признаков на обратимую матрицу не влияет на качество модели, собственно, этим свойством и можно воспользоваться для преобразования. Предложим такой алгоритм: при предположении, что данные состоят из признаков и целевого признака, признаки необходимо умножать на матрицу, а целевой признак оставить как есть.

 Осталось определить, какую матрицу необходимо использовать, чтобы соблюсти корректность всех операций. Все, что нам известно, матрица обратимая.

Подумаем над размером матрицы. Пусть в таблице с признаками K столбцов и Z записей.

На количестве записей концентрироваться не будем - т.к. количество объектов в выборке может меняться. Имеем матрицу: K х Z, которую надо умножить на другую матрицу и получить третью матрицу - которая подается на вход в модель машинного обучения - в качестве признаков. Параметры третьей матрицы, в общем смысле, могут быть любыми - это новые признаки, и модель обучится на любом их количестве, а также сможет по ним предсказывать, но смысл задания 'Совершить ПРЕОБРАЗОВАНИЕ признаков' диктует нам, что признаков должно остаться все также K, но выглядеть они должны по-другому, наверное.

Имеем: K х Z @ (новая матрица) = K x Z.
Отсюда несложно получить - новая матрица должна быть размером K х K.

Получили такой алгоритм: данные следует разбить на признаки и целевой признак, затем - признаки умножить на обратимую матрицу размера К х К, где K - количество признаков, далее - данные можно собрать обратно, если это необходимо, в противном случае - и признаки, и целевой признак можно использовать при создании модели линейной регрессии, а также при предсказаниях с ее помощью.

Матрица, используемая в алгоритме, должна генерироваться случайно, с естественным условием - на одну модель одна матрица.

Важная пометка к алгоритму: признаки, по которым совершаются предсказания, должны быть умножены ровно на ту же самую матрицу, на которую умножались и признаки в обучающей выборке. 

**Вывод по Шагу 3:** итак, на данном шаге мы предложили алгоритм преобразования данных для решения задачи. Обоснование, почему качество линейной регрессии не поменяется в случае его применения, находится в предыдущем шаге.

<h3> Шаг 4: Проверка и программирование выбранного алгоритма</h3>
<a id='alg1'></a>

На данном шаге нам необходимо запрограммирововать выбранный алгоритм, а также - с помощью метрики r2 убедиться, что его качество не отличается до и после преобразований.

Напишем функцию - она принимает на вход датафрейм с признаками и целевым признаком, название столбца с целевым признаком. Возвращает измененный по алгоритму датафрейм. Столбец с целевым признаком в нем присутствует в нетронутом виде. Можно использовать эту функцию для преобразования и при отсутствии значений целевого признака. 

Матрица для шифрования получается случайным образом, но свое значение она сохраняет от вызова к вызову.

In [6]:
def transformation(data, *target_name):
    np.random.seed(12345)
    matrix_to_use = np.random.sample((4,4))
    if len(target_name) == 1:
        target = data[target_name[0]]
        features = data.drop(columns=[target_name[0]])
        result = pd.DataFrame(data = features.values @ matrix_to_use, columns = features.columns)
        result[target_name[0]] = target
        return result
    else:
        return pd.DataFrame(data = data.values @ matrix_to_use, columns = data.columns)

Проверим работу функции на датафрейме целиком.

In [7]:
transformation(data, 'Страховые выплаты').head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,37169.983952,32441.905747,37126.682258,47708.455342,0
1,28484.575966,24863.156593,28457.829124,36560.358528,1
2,15743.503414,13742.238084,15729.981916,20206.383593,0
3,31241.345787,27266.582942,31200.559759,40101.520436,0
4,19563.289157,17075.165256,19542.546847,25108.599330,0


На признаках отдельно.

In [8]:
features = data.drop(columns=['Страховые выплаты'])
target = data['Страховые выплаты']
transformation(features).head()

,Пол,Возраст,Зарплата,Члены семьи
0,37169.983952,32441.905747,37126.682258,47708.455342
1,28484.575966,24863.156593,28457.829124,36560.358528
2,15743.503414,13742.238084,15729.981916,20206.383593
3,31241.345787,27266.582942,31200.559759,40101.520436
4,19563.289157,17075.165256,19542.546847,25108.599330


Все зашифровано. Теперь будем строить модели - на преобразованные и на исходных данных. Качество проверим с помощью r2. Еще один момент перед этим - поделим данные на обучающую и тестовую выборки.

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, 
                                                                           random_state=12345)

Теперь обучим модель на обучающей выборке в исходном виде. Проверим качество на тестовой выборке, которая также в исходном виде.

In [10]:
model1 = LinearRegression()
model1.fit(features_train, target_train)
predicted1 = model1.predict(features_test)
print('R2 модели, обученной на выборке в исходном виде: {:.2}'.format(r2_score(target_test, predicted1)))

R2 модели, обученной на выборке в исходном виде: 0.44


Теперь преобразуем выборки и обучим другую модель - уже на новых вариантах.

In [11]:
features_train_v2 = transformation(features_train)
features_test_v2 = transformation(features_test)

In [12]:
model2 = LinearRegression()
model2.fit(features_train_v2, target_train)
predicted2 = model2.predict(features_test_v2)
print('R2 модели, обученной на преобразованной: {:.2}'.format(r2_score(target_test, predicted2)))

R2 модели, обученной на преобразованной: 0.44


Результат на лицо - качество одинаковое, алгоритм работает.

**Вывод по Шагу 4**: на данном Шаге мы запрограммировали алгоритм, который определили в предыдущем разделе, а также проверили, что он сохраняет качество линейной регрессии.